# 1: Data Structures
In this mission and the next, we're going to dive into some of Pandas' internals to better understand how Pandas does things under the hood.

The three key data structures in Pandas are:

* Series (collection of values)
* DataFrame (collection of Series objects)
* Panel (collection of DataFrame objects)

and we'll be focusing on the Series object in this mission.

Series objects use NumPy arrays for fast computation, but build on them by adding valuable features for analyzing data. For example, while NumPy arrays utilize an integer index, Series objects can utilize other index types, like a string index. Series objects also allow for mixed data types and utilize the NaN Python value for handling missing values.

A Series object can hold many data types, including:

* float - for representing float values
* int - for representing integer values
* bool - for representing Boolean values
* datetime64[ns] - for representing date & time, without time-zone
* datetime64[ns, tz] - for representing date & time, with time-zone
* timedelta[ns] - for representing differences in dates & times (seconds, minutes, etc.)
* category - for representing categorical values
* object - for representing String values

Before we dive further, let's introduce the dataset we'll be working with. The FiveThirtyEight team recently released a dataset containing the critics scores for all movies that have substantive user and critic reviews from IMDB, Rotten Tomatoes, Metacritic, and Fandango. We'll be working with the file fandango_score_comparison.csv, which you can download from their [Github repo](https://github.com/fivethirtyeight/data/tree/master/fandango). Here are some of the columns in the dataset:


In [3]:
import pandas as pd
fandango=pd.read_csv("fandango_score_comparison.csv")
fandango.head(5)

,FILM,RottenTomatoes,RottenTomatoes_User,Metacritic,Metacritic_User,IMDB,Fandango_Stars,Fandango_Ratingvalue,RT_norm,RT_user_norm,...,IMDB_norm,RT_norm_round,RT_user_norm_round,Metacritic_norm_round,Metacritic_user_norm_round,IMDB_norm_round,Metacritic_user_vote_count,IMDB_user_vote_count,Fandango_votes,Fandango_Difference
0,Avengers: Age of Ultron (2015),74,86,66,7.1,7.8,5.0,4.5,3.70,4.3,...,3.90,3.5,4.5,3.5,3.5,4.0,1330,271107,14846,0.5
1,Cinderella (2015),85,80,67,7.5,7.1,5.0,4.5,4.25,4.0,...,3.55,4.5,4.0,3.5,4.0,3.5,249,65709,12640,0.5
2,Ant-Man (2015),80,90,64,8.1,7.8,5.0,4.5,4.00,4.5,...,3.90,4.0,4.5,3.0,4.0,4.0,627,103660,12055,0.5
3,Do You Believe? (2015),18,84,22,4.7,5.4,5.0,4.5,0.90,4.2,...,2.70,1.0,4.0,1.0,2.5,2.5,31,3136,1793,0.5
4,Hot Tub Time Machine 2 (2015),14,28,29,3.4,5.1,3.5,3.0,0.70,1.4,...,2.55,0.5,1.5,1.5,1.5,2.5,88,19560,1021,0.5


# 2: Integer Index
DataFrames use Series objects to represent the columns in the data. When you select a single column from a DataFrame, Pandas will return the Series object representing that column. Each individual Series object in a DataFrame is indexed using the integer data type by default. Each value in the Series has a unique integer index, or position. The integer index is 0-indexed, like most Python data structures, and ranges from 0 to n-1, where n is the number of rows. With an integer index, you can select an individual value in the Series if you know it's position as well as select multiple values by passing in a list of index values (similar to a NumPy array).

For both NumPy arrays and Series objects, you can utilize integer index by using bracket notation to slice and select values. Where Series objects diverge from NumPy arrays, however, is the ability to specify a custom index for the values.

To explore this idea further, let's use two Series objects representing the film names and Rotten Tomatoes scores.

In [14]:
series_film=fandango["FILM"]
print(series_film.head(5))
series_rt=fandango["RottenTomatoes"]
print(series_rt.head(5))
#print(fandango["FILM"][fandango['RottenTomatoes']==100])
#print(fandango.loc[70])

0    Avengers: Age of Ultron (2015)
1                 Cinderella (2015)
2                    Ant-Man (2015)
3            Do You Believe? (2015)
4     Hot Tub Time Machine 2 (2015)
Name: FILM, dtype: object
0    74
1    85
2    80
3    18
4    14
Name: RottenTomatoes, dtype: int64


# 3: Custom Index
Both these Series objects use the same integer indices, which means that the value at index 5, for example, would describe the same film in both Series objects (The Water Diviner (2015)). If we had a movie in mind, we need the integer index corresponding to that movie to look up information about it.

If we were given just these two Series objects and we wanted to look up the Rotten Tomatoes score for Minions (2015) and Leviathan (2014), we'd have to:

* find the integer index corresponding to Minions (2015) in series_film
* look up the value at that integer index from series_rt
* find the integer index corresponding to Leviathan (2014) in series_film
* look up the value at that integer index from series_rt

This becomes especially cumbersome as we scale up the problem to looking up information for a larger number of movies. What we really want is a way to look up the Rotten Tomatoes scores for many movies at a time using just one command (and one Series object). To accomplish this, we need to find a way to move away from using an integer index corresponding to the row number and instead use a string index corresponding to the film name. Then we can utilize bracket notation to just pass in a list of strings matching the film names and get back the Rotten Tomatoes scores:



In [22]:
from pandas import Series
film_names = series_film.values
rt_scores = series_rt.values
series_custom[['Minions (2015)', 'Leviathan (2014)']]

Minions (2015)      54
Leviathan (2014)    99
dtype: int64

# 4: Integer Index Preserved
Even though we specified that the Series object uses a custom, string index, the object still maintains an internal integer index that we can use for selection. In this way, Series objects act both like a dictionary and a list since we can access values using our custom index (like the keys in a dictionary) or the integer index (like the index in a list).

In [25]:
fiveten=series_custom[5:10]
fiveten

The Water Diviner (2015)        63
Irrational Man (2015)           42
Top Five (2014)                 86
Shaun the Sheep Movie (2015)    99
Love & Mercy (2015)             89
dtype: int64

# 5: Reindexing
Reindexing is the Pandas way of modifying the alignment between labels (index) and the data (values). The reindex() method allows you to specify an alternate ordering of the labels (index) for a Series object. This method takes in a list of strings corresponding to the order of labels you'd like for that Series object.

We can use the reindex() method to sort series_custom in alphabetical order by film. To accomplish this, we need to:

* return a list representation of the current index using tolist()
* sort the index using sorted()
* use reindex() to set the new ordered index

The following code cell contains the logic to accomplish the first task and we'll leave it up to you to finish the rest.

In [30]:
original_index = series_custom.index.tolist()
sorted_index=sorted(original_index)
sorted_by_index=series_custom.reindex(sorted_index)
sorted_by_index

'71 (2015)                                         97
5 Flights Up (2015)                                52
A Little Chaos (2015)                              40
A Most Violent Year (2014)                         90
About Elly (2015)                                  97
Aloha (2015)                                       19
American Sniper (2015)                             72
American Ultra (2015)                              46
Amy (2015)                                         97
Annie (2014)                                       27
Ant-Man (2015)                                     80
Avengers: Age of Ultron (2015)                     74
Big Eyes (2014)                                    72
Birdman (2014)                                     92
Black Sea (2015)                                   82
Black or White (2015)                              39
Blackhat (2015)                                    34
Cake (2015)                                        49
Chappie (2015)              

# 6: Sorting
We learned how to sort a Series object by the index using the reindex() method in this mission. Sorting by reindexing can be cumbersome if we want to order by the rating (the values in the Series) instead of by film name or for quick exploratory data analysis. To make sorting easier, Pandas comes with a sort_index() method, which returns a Series sorted by the index, and a sort_values() method method, which returns a Series sorted by the values. Since the values representing the Rotten Tomatoes scores are integers, sorting by values will sort in numerically ascending order (low to high) in our case.

In both cases, the link between each element's index (film name) and value (score) is preserved. This is known as data alignment and is a key tenet of Pandas that is incredibly important when analyzing data. Unless we specifically change a value or an index, Pandas allows us to assume the linking will be preserved.

In [36]:
sc2= series_custom.sort_index(ascending=False)
sc3= series_custom.sort_values(ascending=False)
print(sc2.head(10))
print(sc3.head(10))

Woman in Gold (2015)                52
Wild Tales (2014)                   96
While We're Young (2015)            83
When Marnie Was There (2015)        89
What We Do in the Shadows (2015)    96
Welcome to Me (2015)                71
Vacation (2015)                     27
Unfriended (2015)                   60
Unfinished Business (2015)          11
Unbroken (2014)                     51
dtype: int64
Gett: The Trial of Viviane Amsalem (2015)    100
Seymour: An Introduction (2015)              100
Selma (2014)                                  99
Phoenix (2015)                                99
Song of the Sea (2014)                        99
Leviathan (2014)                              99
Shaun the Sheep Movie (2015)                  99
Timbuktu (2015)                               99
Mr. Turner (2014)                             98
Paddington (2015)                             98
dtype: int64


# 7: Vectorized Operations
There are many cases where you want to transform an entire column in a dataset, since a column is really a vector of values. Series objects have powerful support for vectorized operations, which enable you to run computations over an entire column incredibly quickly. Since Pandas builds on top of NumPy, it takes advantage of NumPy's vectorizaton capabilities which generates incredibly optimized, low level C code (C is another programming language) to loop over the values instead. Using a for loop to iterate over the values in a Series object to run item-wise computations is much slower, especially for larger datasets.

You can use any of the standard Python arithmetic operators (+, -, *, and /) to transform every value in a Series object. For example, if we wanted to transform the Rotten Tomatoes scores from a 0 to 100 point scale to a 0 to 10 scale, we can use the Python / division operator to divide the Series by 10:


In [ ]:
series_custom/10

and we'll get back a new Series object with each value divided by 10. We can even use NumPy functions to transform and run calculations over Series objects:

In [ ]:
# Add each value with each other
np.add(series_custom, series_custom)
# Apply sine function to each value
np.sin(series_custom)
# Return the highest value (will return a single value not a Series)
np.max(series_custom)

The values in a Series object are treated as an ndarray, the core data type in NumPy. Applying some NumPy functions will return a new Series object while others will return just a single value. NumPy's documentation gives you a good sense of the return value for each function. If a particular NumPy function usually returns an ndarray, it will instead return a Series object when applied to a Series.

The original DataFrame contains the column RT_norm, which represents a normalized score (from 0 to 5) of the Rotten Tomatoes average critics score. Let's use vectorized operations to normalize series_custom back to the 0-5 scale.

In [49]:
import numpy as np
series_normalized=series_custom/20
#series_normalized.sort_values(ascending=False)

# 8: Comparing And Filtering
Pandas utilizes vectorized operations everywhere, including when filtering values within a single Series object or comparing 2 different Series objects. For example, to find all films that had above a 50 Rotten Tomatoes average critics rating, running:



series_custom > 50
will actually return a Series object with a boolean value for each film. This is because the filter (> 50) is applied to each value in the Series object. To retrieve the actual films a Series object containing just the films with a rating greater than 50, we need to pass in this Boolean series into the original Series object.



series_greater_than_50 = series_custom[series_custom > 50]
To help make it easy to separate complex comparison and filtering logic into modular pieces, Pandas returns Boolean Series objects as the intermediate representation of the logic. We can specify filtering criteria in different variables and chain them together using the & operator, which represents and, as well as the | operator, representing or. Finally, we can utilize a Series object's bracket notation to pass in an expression representing a Boolean Series object to get back the filtered dataset.

In [48]:
criteria_one = series_custom > 50
criteria_two = series_custom < 75
both_criteria=series_custom[criteria_one & criteria_two]

# 9: Alignment
One of the core tenets of Pandas is data alignment. Series objects align along indices and DataFrame objects align along both indices and columns. This means that for Series objects, the link between the index labels and the actual values is implicitly preserved across operations and transformations unless we explicitly break the link. For DataFrame objects, the values are linked to the index labels and the column labels, and are also preserved unless we explicitly break the link (by reassigning or editing a column or index label, for example).

This core tenet allows us to use Pandas effectively when working with data and is a big advantages over just using NumPy objects. For Series objects in particular, this means we can use the standard Python arithmetic operators (+, -, *, and /) to add, subtract, multiple, and divide the values at each index label for 2 different Series objects.

We can take advantage of this functionality to calculate the mean of the Rotten Tomatoes' critics average rating and the Rotten Tomatoes' average user rating.

In [60]:
rt_critics = Series(fandango['RottenTomatoes'].values, index=fandango['FILM'])
rt_users = Series(fandango['RottenTomatoes_User'].values, index=fandango['FILM'])
rt_mean=(rt_critics+rt_users)/2
adel_series=fandango["RottenTomatoes"]
type(adel_series)

pandas.core.series.Series